In [1]:
import numpy as np
import pandas as pd
from rdkit.Chem import MolStandardize

RDKit WARNING: [16:13:18] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
def mol2alt_sentence(mol, radius):
    """Same as mol2sentence() expect it only returns the alternating sentence
    Calculates ECFP (Morgan fingerprint) and returns identifiers of substructures as 'sentence' (string).
    Returns a tuple with 1) a list with sentence for each radius and 2) a sentence with identifiers from all radii
    combined.
    NOTE: Words are ALWAYS reordered according to atom order in the input mol object.
    NOTE: Due to the way how Morgan FPs are generated, number of identifiers at each radius is smaller

    Parameters
    ----------
    mol : rdkit.Chem.rdchem.Mol
    radius : float
        Fingerprint radius

    Returns
    -------
    list
        alternating sentence
    combined
    """
    radii = list(range(int(radius) + 1))
    info = {}
    _ = AllChem.GetMorganFingerprint(mol, radius, bitInfo=info)  # info: dictionary identifier, atom_idx, radius

    mol_atoms = [a.GetIdx() for a in mol.GetAtoms()]

    #     print(mol_atoms)
    dict_atoms = {x: {r: None for r in radii} for x in mol_atoms}

    for element in info:
        for atom_idx, radius_at in info[element]:
            dict_atoms[atom_idx][radius_at] = element  # {atom number: {fp radius: identifier}}

    # merge identifiers alternating radius to sentence: atom 0 radius0, atom 0 radius 1, etc.
    identifiers_alt = []
    for atom in dict_atoms:  # iterate over atoms
        for r in radii:  # iterate over radii
            identifiers_alt.append(dict_atoms[atom][r])

    alternating_sentence = map(str, [x for x in identifiers_alt if x])

    return list(alternating_sentence)

In [3]:
allowedAtomsDict = {
    'H' : 1,'h' : 0,
    'B' : 5,'b' : 0,
    'C' : 6,'c' : 0,
    'N' : 7,'n' : 0,
    'O' : 8,'o' : 0,
    'F' : 9,'f' : 0,
    'P' : 15,'p': 0,
    'S' : 16,'s': 0,
    'Cl': 17,'Br' : 35
}


In [4]:
word = "AaBbCcDdEeFfGgHhIiJjKkLlMmNnOoPpQqRrSsTtUuVvWwXxYyZzBrCl"
def isValidCharacter(c):
    if c not in word or (c in word and c in "HhBbCcNnOoFfPpSsClBr"):
        return True
    return False

def isValidSmiles(smiles,atom_weight = 600,heavy_atom_count = 50):
    '''
        1. smiles能够被rdkit包处理
        2. smiles只包含特定元素
        3. smiles原子权重
    '''
    t_weight = 0
    heavyAtomCount = 0
    left = -len(smiles)-1
    right = -1
    idx = -1
    while True:
        if idx <= left:
            break
        c = smiles[idx]
        if smiles[idx] == 'r' or smiles[idx] == 'l' :
            c = (smiles[idx-1] if idx -1 > right else "#") + c
            idx = idx - 1
        idx = idx - 1
        if isValidCharacter(c) == True:
            if c in allowedAtomsDict.keys():
                t_weight = t_weight + int(allowedAtomsDict[c])
                heavyAtomCount = heavyAtomCount + (1 if int(allowedAtomsDict[c]) > 1 else 0)
        else:
            return False
#     print(type(t_weight),ttype(heavy_atom_count))
    return  True if t_weight >= 3 and t_weight <= atom_weight and heavyAtomCount <= heavy_atom_count else False

In [5]:
def standardizeAndcanonical(smi):
    lfc = MolStandardize.fragment.LargestFragmentChooser()
    # standardize
    mol = Chem.MolFromSmiles(smi)
    mol2 = lfc.choose(mol)
    smi2 = Chem.MolToSmiles(mol2)
    #     print(smi2)
    #     # canonical
    #     can_smi = Chem.MolToSmiles(Chem.MolFromSmiles(smi2))
    # #     print(can_smi)
    #     print(can_smi == smi2)
    return smi2

In [6]:
import csv
import pandas as pd
import random
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
import seaborn as sns
from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg
import pandas as pd


In [7]:
# 预先加载数据集
df = pd.read_csv('raw/Lipophilicity.csv',sep=',')
# print(df)
smiles_list = []
# line = line.rstrip().split(',')
# smiles = str(line[8])
# val = float(line[7]
for _,smlAndlabel in df.iterrows():
    smiles, label = smlAndlabel[2], smlAndlabel[1]
    smiles_list.append([smiles,label])
# print(smiles_list)

In [8]:
def generate_scaffold_file( smiles_list, num=0,):
    new_smiles_list = []
    for i in range(len(smiles_list)):
        smiles = smiles_list[i][0]
        label = smiles_list[i][1]
        smi = standardizeAndcanonical(smiles)
        if isValidSmiles(smi) == True:
            t = Chem.MolFromSmiles(smi)
            if t != None:
                sentence = mol2alt_sentence(t,1)
#                     print(sentence[0] == 'none')
                if len(sentence) % 2 == 0:
                        new_smiles_list.append([smiles, label])
    idx = [i for i in range(len(new_smiles_list))]
    random.shuffle(idx)
    idx_train, idx_validation, idx_test = idx[0:int(len(idx)*0.8)],idx[int(len(idx)*0.8):int(len(idx)*0.9)]\
    ,idx[int(len(idx)*0.9):]
    print(len(idx_train), len(idx_validation), len(idx_test), len(idx_train)+len(idx_validation)+len(idx_test))
    with open("train_" + str(num) + ".txt","w") as f:
        for id in idx_train:
            smiles = smiles_list[id][0]
            label = smiles_list[id][1]
            f.write(smiles+","+str(label)+"\n")
            
    with open("validation_" + str(num) + ".txt","w") as f:
        for id in idx_validation:
            smiles = smiles_list[id][0]
            label = smiles_list[id][1]
            f.write(smiles+","+str(label)+"\n")
            
    with open("test_" + str(num) + ".txt","w") as f:
        for id in idx_test:
            smiles = smiles_list[id][0]
            label = smiles_list[id][1]
            f.write(smiles+","+str(label)+"\n")
    

In [9]:
generate_scaffold_file(smiles_list,0)
generate_scaffold_file(smiles_list,1)
generate_scaffold_file(smiles_list,2)
generate_scaffold_file(smiles_list,3)
generate_scaffold_file(smiles_list,4)
generate_scaffold_file(smiles_list,5)
# generate_scaffold_file("split-random-1.npy",1,smiles_list)
# generate_scaffold_file("split-random-2.npy",2,smiles_list)
# generate_scaffold_file("split-random-3.npy",3,smiles_list)
# generate_scaffold_file("split-random-4.npy",4,smiles_list)
# generate_scaffold_file("split-random-5.npy",5,smiles_list)

3332 417 417 4166
3332 417 417 4166
3332 417 417 4166
3332 417 417 4166
3332 417 417 4166
3332 417 417 4166
